In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

import pandas as pd
import numpy as np
import os
import glob

In [2]:
# DRONE_NAMES = { 
#     "Phantom" : "Phantom",
#     "Ruko_F11_Pro" : "Ruko_F11_Pro",
#     "Mavic_Air_2_" : "Mavic_Air_2",
#     "Mavic_Air_2S" : "Mavic_Air_2S",
#     "DeerC" : "DeerC",
#     "Mini_SE" : "Mini_SE",
#     "test" : "test",
# }

def get_label_from_filename(filename): 
    if "Phantom" in filename: return "Phantom"
    if "Ruko_F11_Pro" in filename: return "Ruko_F11_Pro"
    if "Mavic_Air_2_" in filename: return "Mavic_Air_2"
    if "Mavic_Air_2S" in filename: return "Mavic_Air_2S"
    if "DeerC" in filename: return "DeerC"
    if "Mini_SE" in filename: return "Mini_SE"
    if "test" in filename: return "test"
    
def load_iq(file, window_size, samples_per_file): 
    df = pd.read_csv(file, nrows=window_size * samples_per_file)
    iq = df[['i', 'q']].to_numpy()
    return iq
    

In [3]:
# Path to your IQ files
drone_files = "./data/*.csv"
target_len = 500000
X = []
y = []

# Example: 2048-sample windows with 50% overlap
window_size = 2048 # number of iq samples to chunk
samples_per_file = 1000
step = 1024
segments = []
labels = []


for file in glob.glob(drone_files):
    # print("The current file: ", file)
    label = get_label_from_filename(file)
    # print("Label: ", label)
    iq = load_iq(file, window_size, samples_per_file)  # shape (N, 2)
    # print("iq:\n", iq)
    for i in range(0, len(iq)-window_size, step):
        window = iq[i:i+window_size].flatten()
        segments.append(window)
        labels.append(label)
    # print("Segments:\n", segments)
    # print("Labels:\n", labels)
    

X = [segment.flatten() for segment in segments]
y = labels

# print("X feature:\n", X)
# print("y feature:\n", y)

X = np.array(X)
y = np.array(y)

In [4]:
# for sample, label in zip(X, np.unique(y)): 
#     print("X feature:\n", sample)
#     print("IQ samples in X feature: ", int(sample.size / 2))
#     print("y label: ", label)
#     print('-------------------------------------------')

## Support Vector Classifier 

Training & fitting

In [ ]:
# Train/test split (e.g. 80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Standardize features (zero mean, unit variance)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


SVCModel = SVC(kernel='rbf', C=10, gamma='scale')  # tweak C/gamma as needed
SVCModel.fit(X_train, y_train)

Prediction and metrics

In [ ]:

y_pred = SVCModel.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


## Random Forest Classifier